In [1]:
from bindsnet.network import load_network
from bindsnet.encoding import poisson
from bindsnet.analysis.plotting import plot_weights
from IPython.display import clear_output
import matplotlib.pyplot as plt
import pickle
import torch
from encoder import *
import time as T
plt.rcParams["figure.figsize"] = (20, 20)
lfname = 'logs/' + str(int(T.time())) + '_anomaly.txt'

In [2]:
track = pickle.load(open('encoding/track3_numenta.p', 'rb'))*255

In [3]:
network = load_network('trained.net')
network.connections[('X', 'Ae')].update_rule = None
exc_monitor = network.monitors['exc']

In [4]:
time = 500

In [5]:
def log(msg):
    print(msg, end='')
    with open(lfname, 'a') as f:
        f.write(msg)

In [6]:
def detect(spikes):
    cols = torch.sum(spikes, dim=1)
    for i, v in enumerate(cols):
        if v != 0:
            log(f'Neuron: {i} Spikes: {int(v)}\n')

In [7]:
for i in range(4, len(track)):
    orig = torch.cat((track[i-4], track[i-3], track[i-2], track[i-1], track[i]))
    pt = poisson(orig, time)
    
    inpts = {'X': pt}
    network.run(inpts=inpts, time=time)
    spikes = exc_monitor.get('s')
    # network.reset_()

    log(f'Iteration {i - 3}\n')
    detect(spikes)
    
    fig = plt.figure(figsize=(20, 20))
    plt.subplot(2, 2, 1)
    plt.imshow(spikes, cmap='binary')
    plt.subplot(2, 2, 2)
    plt.imshow(orig.view(125, 40), cmap='gist_gray')
    plt.show()
    
    clear_output(wait=True)

Iteration 285
Neuron: 5 Spikes: 2
Neuron: 52 Spikes: 1
Neuron: 55 Spikes: 3
Neuron: 58 Spikes: 2
Neuron: 72 Spikes: 2
Neuron: 73 Spikes: 1
Neuron: 77 Spikes: 2
Neuron: 80 Spikes: 1
Neuron: 92 Spikes: 1


KeyboardInterrupt: 

In [ ]:
network.connections[('X', 'Ae')].w.shape

In [ ]:
plot_weights(network.connections[('X', 'Ae')].w)